In [4]:
import random 
from platypus import *
import pandas as pd 

In [3]:
tempo = [3,15]
assets = dict()
assets[0] = {
    "prezzo": 750,
    "quote": 900,
    "quoteMinime": 30,
    "quoteMassime": 1000
}
assets[1] = {
    "prezzo": 1000,
    "quote": 1000,
    "quoteMinime": 30
}

r = [
    [0.02, 0.07], 
    [0.05, 0.12]
]
sigma = [
    [0.04, 0.13], 
    [0.06, 0.15]
]

corr = [
    [
        [1,1],[0.5,0.4]
    ],
    [
        [0.5,0.4],[1,1]
    ],
]
val = 0
for asset in assets:
    val+= assets[asset]["prezzo"]*assets[asset]["quote"]
print "Valore port:", val
def funct(x):
    valP = []
    pesi = []
    for i in range(0,2):
        row=x[i*len(r[0]):(i+1)*len(r[0])]
        sumx = 0
        for j in range(0,len(row)):
            sumx+= assets[j]["prezzo"]*row[j]
        y = []
        valP.append(sumx)
        for j in range(0,len(row)):
            y.append(row[j]*assets[j]["prezzo"]/sumx)
        pesi.append(y)
    #print "Pesi:", pesi,"\nQuote:", x
   
    vincoli = []

    for j in range(0,2):
        z = 0
        #print valP[j]
        for i in range(0,len(r[0])):
            #print r[j][i], x[i+j*len(r[0])]
            #print x[i+j*len(r[0])],sum_xi[j] , x[j*len(r[0]):(j+1)*len(r[0])]
            #print i,j,r[i][j],pesi[j][i],r[i][j]*pesi[j][i]
            z+= r[i][j]*pesi[j][i]
        #print z , z/sum_xi[j], sum_xi[j]
        #print "**********"
        vincoli.append(round((1+z)**tempo[j] *valP[j],0))
        vincoli.append(round((1+z)**tempo[j] *valP[j],0))
    #print "Vincoli",vincoli
    funzioni = []
    #print "x", x
    for j in range(0,2):
        y1 = 0
        for i in range(0,len(r[0])):
            #print "y1", j, i, i+j*len(r[0]), x[i+j*len(r[0])]
            y1+= pesi[j][i]**2 * sigma[j][i]**2

        y2 = 0
        for i in range(0,len(r[0])):
            for k in range(0,len(r[0])):
                if i != k:
                   #print "y2", j, k, i, i+j*len(r[0]), k+j*len(r[0])
                    y2 += pesi[j][i] * pesi[j][k] * sigma[j][i] * sigma[j][k] * corr[i][k][j]
        funzioni.append(y1+y2)
    for i in range(0,len(r[0])):
        s = 0
        for j in range(0,2):
            #print "s", i,j, i+j*len(r[0]) ,x[i+j*len(r[0])]
            s+= x[i+j*len(r[0])]
        vincoli.append(round(s,2)) 
       # print funzioni, vincoli, x
    return funzioni , vincoli

problem = Problem(4, 2, 6)
problem.types[:] = Real(30, 10000)
problem.constraints[0] = ">= 90000"
problem.constraints[1] = "<= 110000"
problem.constraints[2] = ">= 1900000"
problem.constraints[3] = "<= 2100000"
problem.constraints[4] = "<= 900"
problem.constraints[5] = "<= 1000"
problem.function = funct

algorithms = [ 
    #NSGAII(problem),
    #SPEA2(problem),
    #GDE3(problem),
    #MOEAD(problem),
    #SMPSO(problem),
    #CMAES(problem),
    PAES(problem),
]
for algorithm in algorithms:
    algorithm.run(500)

for algorithm in algorithms:
    print algorithm.result[0].constraints
    print algorithm.result[0].variables
    print algorithm.result[0].objectives
    
i=0
objectives = pd.DataFrame()
for algorithm in algorithms:
    i +=1
    feasible_solutions = [s for s in algorithm.result if s.feasible]
    if len(feasible_solutions) > 0:
        print i
       # print("Final solutions feasible:")
        x = feasible_solutions[0].objectives
        y = {v: round(k,3) for v, k in enumerate(x)}
        y[len(y)+1] = i
        objectives = objectives.append(y, ignore_index = True)
        print feasible_solutions[0].constraints
        print feasible_solutions[0].variables
        print feasible_solutions[0].objectives
   # else:
        #print("Final solutions:")
        #objectives[i] = algorithm.result[0].objectives
        #print algorithm.result[0].constraints
#print objectives
print objectives.sort_values([0,1], ascending=[True, True])

Valore port: 1675000
[95572.0, 95572.0, 1901517.0, 1901517.0, 895.25, 73.78]
[76.51490330432792, 30.00000181756454, 818.732383268319, 43.78220057633335]
[0.00385411162431, 0.00368371969956]
1
[95572.0, 95572.0, 1901517.0, 1901517.0, 895.25, 73.78]
[76.51490330432792, 30.00000181756454, 818.732383268319, 43.78220057633335]
[0.00385411162431, 0.00368371969956]
       0      1    3
0  0.004  0.004  1.0


In [32]:
def objectives_allocation(assets_risk, asset_return, corr, objectives, assets):
    def funct(x):
        subportfolio_values = []
        weights = []
        for i in range(0,len(objectives)):
            row=x[i*len(assets):(i+1)*len(assets)]
            subportfolio_value = 0
    
            for j in range(0,len(row)):
                subportfolio_value+= assets[j]["price"]*row[j]
            weight = []
            subportfolio_values.append(subportfolio_value)
            for j in range(0,len(row)):
                #print j
                weight.append(row[j]*assets[j]["price"]/subportfolio_value)
            weights.append(weight)
  #      print "Pesi:", weights,"\nQuote:", x

        constraints = []
        for j in range(0,len(objectives)):
            #print j
            z = 0
            #print valP[j]
            for i in range(0,len(assets)):
                #print r[j][i], x[i+j*len(r[0])]
                #print x[i+j*len(r[0])],sum_xi[j] , x[j*len(r[0]):(j+1)*len(r[0])]
                #print i,j,r[i][j],pesi[j][i],r[i][j]*pesi[j][i]
                z+= assets_return[i][j]*weights[j][i]
            #print z , z/sum_xi[j], sum_xi[j]
            #print "**********"
            #print obk, (1+z), objectives[obk]["time"] ,subportfolio_values[j]
            constraints.append(round((1+z)**objectives[j]["time"] *subportfolio_values[j],0))
            constraints.append(round((1+z)**objectives[j]["time"] *subportfolio_values[j],0))
        #print "Vincoli",vincoli
        funzioni = []
        #print "x", x
        for j in range(0,len(objectives)):
            y1 = 0
            for i in range(0,len(assets)):
                #print "y1", j, i, i+j*len(r[0]), x[i+j*len(r[0])]
                y1+= weights[j][i]**2 * assets_risk[j][i]**2

            y2 = 0
            for i in range(0,len(assets)):
                for k in range(0,len(assets)):
                    if i != k:
                       #print "y2", j, k, i, i+j*len(r[0]), k+j*len(r[0])
                        y2 += weights[j][i] * weights[j][k] * assets_risk[j][i] * assets_risk[j][k] * corr[i][k][j]
            funzioni.append(y1+y2)
        for i in range(0,len(assets)):
            s = 0
            for j in range(0,2):
                #print "s", i,j, i+j*len(r[0]) ,x[i+j*len(r[0])]
                s+= x[i+j*len(assets)]
            constraints.append(round(s,2)) 
           # print funzioni, vincoli, x
        return funzioni , constraints

    problem = Problem(4, 2, 6)
    problem.types[:] = Real(10,1000)
    #problem.types[0] = Real(assets[0]["min_quote"], assets[0]["max_quote"])
    #problem.types[1] = Real(assets[1]["min_quote"], assets[1]["max_quote"])
    problem.constraints[0] = ">= "+ str(objectives[0]["value"]*0.9)
    problem.constraints[1] = "<= "+ str(objectives[0]["value"]*1.1)
    problem.constraints[2] = ">= "+ str(objectives[1]["value"]*0.9)
    problem.constraints[3] = "<= "+ str(objectives[1]["value"]*1.1)
    problem.constraints[4] = "<= "+ str(assets[0]["max_quote"])
    problem.constraints[5] = "<= "+ str(assets[1]["max_quote"])
    problem.function = funct

    algorithms = [ 
        #NSGAII(problem),
        #SPEA2(problem),
        #GDE3(problem),
        #MOEAD(problem),
        #SMPSO(problem),
        #CMAES(problem),
        PAES(problem),
    ]
    for algorithm in algorithms:
        algorithm.run(500)

    for algorithm in algorithms:
        print algorithm.result[0].constraints
        print algorithm.result[0].variables
        print algorithm.result[0].objectives

    i=0
    objectives = pd.DataFrame()
    for algorithm in algorithms:
        i +=1
        feasible_solutions = [s for s in algorithm.result if s.feasible]
        if len(feasible_solutions) > 0:
           # print("Final solutions feasible:")
            x = feasible_solutions[0].objectives
            y = {v: round(k,3) for v, k in enumerate(x)}
            y[len(y)+1] = i
            objectives = objectives.append(y, ignore_index = True)
            print feasible_solutions[0].constraints
            print feasible_solutions[0].variables
            print feasible_solutions[0].objectives
       # else:
            #print("Final solutions:")
            #objectives[i] = algorithm.result[0].objectives
            #print algorithm.result[0].constraints
    #print objectives
    print objectives.sort_values([0,1], ascending=[True, True])

In [33]:
tempo = [3,15]
assets = [
    {
    "price": 7500,
    "min_quote": 30,
    "max_quote": 1000
    },
    {
    "price": 1000,
    "min_quote": 30,
    "max_quote": 1000
    } 
] 
assets_return = [
    [0.02, 0.07], 
    [0.05, 0.12]
]
assets_risk = [
    [0.04, 0.13], 
    [0.06, 0.15]
]

corr = [
    [
        [1,1],[0.5,0.4]
    ],
    [
        [0.5,0.4],[1,1]
    ],
]

objectives=[ {
    "time": 10,
    "value": 2000000,
},
{
    "time": 3,
    "value": 200000,
}]
val = 0
for i in range(0,len(assets)):
    val+= assets[i]["price"]*assets[i]["max_quote"]
print "Valore port:", val


objectives_allocation(assets_risk, assets_return, corr, objectives, assets)

Valore port: 8500000
[2182007.0, 2182007.0, 209625.0, 209625.0, 258.23, 20.03]
[236.93568160433813, 10.034486829090516, 21.294795519309897, 10.000000000000917]
[0.00161164955306, 0.00366562087109]
[2182007.0, 2182007.0, 209625.0, 209625.0, 258.23, 20.03]
[236.93568160433813, 10.034486829090516, 21.294795519309897, 10.000000000000917]
[0.00161164955306, 0.00366562087109]
       0      1    3
0  0.002  0.004  1.0
